In [1]:
from sentence_transformers import SentenceTransformer
import json 
import chromadb
from chromadb.utils.embedding_functions import EmbeddingFunction

/home/user/repos/rag-system/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class SentenceTransformerEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def __call__(self, texts: list[str]) -> list[list[float]]:
        return self.model.encode(texts, task='retrieval.passage', batch_size=1, show_progress_bar=True).tolist()


In [3]:
documents = json.load(open("documents/documents.json"))['documents']
texts = [f"{d['title']}\n{d['text']}" for d in documents]

print(documents)

[{'id': 0, 'title': 'Как изменить форму заказа товаров для клиентов?', 'url': 'https://nethouse.ru/about/instructions/kak_izmenit_formu_zakaza', 'text': '1. Перейдите в раздел "Магазин" - "Настройки" - блок "Заказы".\n\t2. Вы можете изменить названия полей, сделать их обязательными/необязательными, поменять местами, добавить новые поля (большие и маленькие), чекбоксы, а также возможность прикрепить файл.\n\tПримечание.\n\tПри выборе размера поля "Мал.поле" оно будет иметь вид:\n\tПри выборе размера поля "Бол.поле" оно будет значительно больше. Кроме того, клиент сможет изменить его, потянув за правый нижний угол.\n\tЧек-бокс ("флажок", "галочка") - здесь можно указать какую-то информацию, с которой посетитель сайта должен согласиться или просто ознакомиться (например, публичная оферта или согласие на обработку персональных данных).\n\tФайл - добавление к форме возможности прикрепить файл.\n\tЗвездочка (*). Если она напротив поля серого цвета, то поле необязательное к заполнению. Если к

In [ ]:
client = chromadb.PersistentClient(path="./chroma_db")
embedding_fn = SentenceTransformerEmbeddingFunction("jinaai/jina-embeddings-v3")

collection = client.get_or_create_collection(
        name="documents-jina",
        embedding_function=embedding_fn
    )

`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


In [2]:
client.get_collection(name="nfkjs")

NameError: name 'client' is not defined

In [5]:
collection.add(
    ids=[str(d["id"]) for d in documents],
    documents=texts,
    metadatas=documents
)

Batches:   0%|          | 0/444 [00:00<?, ?it/s]

Batches: 100%|██████████| 444/444 [36:40<00:00,  4.96s/it] 


In [7]:
collection.query(query_texts=["чат"])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.87s/it]


{'ids': [['279', '79', '73', '48', '1', '306', '406', '202', '382', '51']],
 'embeddings': None,
 'documents': [['Как установить онлайн-чат от Т-Банк?\nПодключение:\n\n\t1. Установите Google Tag Manager по инструкции.\xa0\n\n\t2. Откройте GTM https://tagmanager.google.com.\n\n\t3. Перейдите в раздел «Теги» и нажмите кнопку «Создать».\n\n\t4. Укажите название тега, например, «Т-Банк чат на сайте», нажмите на поле «Конфигурация тега» и выберите «Пользовательский html».\n\n\t5. В настройках тега в поле HTML необходимо вставить код чата от Т-Банк. Зарегистрируйтесь либо авторизуйтесь на сайте сервиса и нажмите кнопку "Скопировать код".\n\n\t7. Вставьте полученный код в настройках тега Google Tag Manager. Далее нажмите на поле «Триггеры».\n\n\t8. Выберите «All Pages» (Все страницы).\n\n\tНажмите кнопку "Сохранить"\n\n\t9. Тег чата для сайта от Т-Банк создан, теперь его нужно опубликовать на сайте. Для этого нажмите кнопку «Отправить» - «Опубликовать» - «Далее».\n\n\t10. Готово! Чат установл

In [9]:
collection.get(ids='406', include=["metadatas"])

{'ids': ['406'],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'included': ['metadatas'],
 'data': None,
 'metadatas': [{'title': 'Privacy Policy',
   'text': '@page { margin: 2cm }\n\t\tp { margin-bottom: 0.25cm; direction: ltr; line-height: 115%; text-align: left; orphans: 2; widows: 2 }\n\t\ta:link { color: #0563c1 }\n\n\tAdministration of the nethouse.ru website is obliged to maintain your privacy on the Internet. We pay great attention to securing the data that you have provided to us. Our privacy policy is based on the General Data Protection Regulation (GDPR) of the European Union. We collect your personal data for such purposes as: improvement of our service, communication with visitors of this site, newsletters, presenting the information requested by the user and for other actions listed below.\n\n\tThere is the data protection center created to protect data users at the nethouse.ru website. All the necessary information about users data management is located in a 

In [4]:
from src.indexing.indexer import DocumentIndexer
from src.indexing.config import IndexingConfig

# Ollama API
indexer_ollama = DocumentIndexer(
    embedding_provider=IndexingConfig.embedding_provider,
    embedding_model=IndexingConfig.ollama_model,
    base_url=IndexingConfig.ollama_base_url,
    collection_name=IndexingConfig.collection_name
)


In [7]:
client = chromadb.PersistentClient(path="storage/chroma_db")

In [8]:
client.count_collections()

2

In [9]:
collection = client.get_collection("documents-test")

In [11]:
from src.indexing.indexer import DocumentIndexer
from src.indexing.config import IndexingConfig, EmbeddingProviderType

indexer = DocumentIndexer(
    embedding_provider=EmbeddingProviderType.OLLAMA,
    embedding_model=IndexingConfig.ollama_model,
    chroma_db_path=IndexingConfig.chroma_db_path,
    collection_name=IndexingConfig.collection_name,
    base_url=IndexingConfig.ollama_base_url
)

In [13]:
indexer.db_connector.get_collection_info()

{'name': 'documents-test', 'count': 444, 'metadata': None}

In [18]:
results = indexer.db_connector.collection.query(
    query_texts=["чат"]
)

In [19]:
for i, (doc_id, document, distance) in enumerate(zip(
    results['ids'][0], 
    results['documents'][0], 
    results['distances'][0]
)):
    print(f"\n{i+1}. ID: {doc_id}, Расстояние: {distance:.4f}")
    print(f"Документ: {document[:200]}...")


1. ID: 66, Расстояние: 1.1263
Документ: Как работает поиск по каталогу товаров?
1. Поиск по каталогу товаров производится по названию, описанию (краткому и полному) и артикулам.

	2. При поиске учитываются только большие и маленькие латинск...

2. ID: 280, Расстояние: 1.1665
Документ: Как разбить статью на две части (убрать часть текста под кат)?
1. Перейдите в редактирование статьи.

2. Поставьте курсор в том месте, где необходимо разделить текст на 2 части. Нажмите кнопку "Добави...

3. ID: 429, Расстояние: 1.1704
Документ: Наполнение каталога товаров с помощью сервиса Cloudparser.ru
"Облачный парсер" - это сервис, с помощью которого можно в несколько кликов получить товары поставщика в формате YML и загрузить их в свой ...

4. ID: 270, Расстояние: 1.1777
Документ: Синхронизация с МоимСкладом: если есть сайт в Nethouse и нет аккаунта в МоемСкладе
1. Подключите платный тариф на сайте.

	2. Перейдите в раздел Приложения - МойСклад.

	3. Во вкладке "Подключение" пр...

5. ID: 38, Расст

In [5]:
client.delete_collection("documents-test")